<a href="https://colab.research.google.com/github/GuillermoFidalgo/QKDP/blob/master/BB84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code for BB84  protocol

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import random
import string
import os

#Function for assigning Standard (s) or Hadamard (h) measurement basis
def randomString(stringLength=2):
    """ Generate a random string of fixed length """
    basis = 'sh'
    return ''.join(random.choice(basis) for i in range(stringLength))

#Function for executing the BB84 protocol using n qubits and N check-bits
def BB84(n,N,Eve=True,Strings=False):
    """    
    BB84(n,N)
    
    n: Number of bits to be used for the key
    
    N: Number of bits to be checked
    
    Eve: Default True. If True, Eve will be present in the protocol. If False, Eve will not be present.
    
    Stings: Default False. If True, return Alice's , Bob's and Eve's:
    1- initial bit strings
    2- keys
    3- initial basis used
    4- check bit sequence
    
    --------
    
    Returns
    
    R: List of strings of "OK" and "ABORT" that indicate when Eve has been detected
    
    a: List of Alice's bits
    b: List of Bob's bits
    e: List of Eve's bits
    x: List of Alice's key
    y: List of Bob's key
    z: List of Eve's key
    
    aa: List of Alice's bases assignments
    bb: List of Bob's bases assignments
    ee: List of Eve's bases assignments
    
    xx: List of Alice's check-bits
    yy: List of Bob's check-bits
    
    """
    
    a=[]
    b=[]
    e=[]
    x=[]
    y=[]
    z=[]
    
    aa=randomString(n)  #Alice's bases assignment
    bb=randomString(n)  #Bob's bases assignment
    if Eve==True:
        ee=randomString(n)  #Eve's bases assignment (when present)
    else:
        ee=aa  #When Eve's not present, she can be thought of as being present, 
               #but having exactly the same bit-string and same basis as Alice
    
    #Generate a,b,e and x,y,z
    for i in range(n):
        a.append(random.randint(0,1))
        if ee[i]==aa[i]:
            e.append(a[i])
        else:
            e.append(random.randint(0,1))
        if bb[i]==ee[i]:
            b.append(e[i])
        else:
            b.append(random.randint(0,1))
        if aa[i]==bb[i]:
            x.append(a[i])
            y.append(b[i])
            z.append(e[i])

    R=[]

    for j in N:
        if j<=len(x):
            s=random.sample(range(len(x)),j)  #Choice of check-bits
            xx=[]
            yy=[]
            for i in range(j):  #Generate xx,yy
                xx.append(x[s[i]])
                yy.append(y[s[i]])
            if xx!=yy:  #Check for Eve's presence
                R.append('ABORT')  #Eve detected
            else:
                R.append('OK')     #Eve not detected
        else:
            break
    
    if Strings==False:
        return R
    if Strings==True:
        return R,a,b,e,aa,bb,ee,x,y,z,xx,yy,s

# One instance of BB84

In [0]:
R,a,b,e,aa,bb,ee,x,y,z,xx,yy,s=BB84(128,range(3),Strings=True)
print("Alice's Check sequence: ",xx)
print("Bob's Check sequence:   ",yy)
print('qubits checked are # :  ',s)
print("Result of Simulation: ",R[-1])

In [0]:
print("Alice's keys: ",x)
print()
print("Bobs's keys: ",y)
print()
print("Eve's keys: ",z)
print()
print("Alice's Basis: ",aa)
print()
print("Bobs's Basis: ",bb)
print()
print("Eve's Basis: ",ee)

# Simulation of the BB84 protocol

In [0]:
k1=100 #Number of iterations of BB84
k2=100 # Sample points 
n=128 #Number of qubits

a=np.arange(5)# dummy variable 
N=2**a #Number of check-bits

# In order to see the simulation with data for all values of possible checkbits uncomment the next line
N=np.arange(1,16)

In [0]:
dist=np.empty([k2,len(N)]) #Probability distribution

#Generate dist,avrg
for j in range(k2):  #Loop for generating dist
    abort=np.zeros(len(N),int) #Number of ABORT
    for i in range(k1):  #Loop for executing BB84
        R=BB84(n,N)
        for m in range(len(R)): #Loop for each N
            if R[m]=='ABORT': #Check for ABORT results
                abort[m]+=1
    pabort=abort/k1  #Experimental probability of ABORT
    dist[j]=pabort

avrg=np.mean(dist,axis=0) #Average of each column of dist

# Now we use Matplotlib's hist function to draw a distribution 

and see what is the average value of finding Eve for the given parameters.



## Configuration of the plots

In [0]:
#Where to store the plots
outpath='plots_BB84'

#Check if folder exists
if outpath not in os.listdir():
    os.mkdir(outpath)
else: 
    print(outpath,'already exists!')


#Configuration for the plots
start = 0
stop  = 1
step  = .05
bins=np.linspace(start, stop, num=250)
    

## For one plot

In [0]:
    
#Making 1 single plot
#Specifing qb changes the plot

qb=1
# qb is a power of 2 (2^qb) = the amount of qubits to inspect UNLESS you uncommented the Line at the simulation stage



# plt.figure(num=qb,figsize=(3,3))
plt.figure(num=qb)
count,val,_=plt.hist(dist[:,qb],bins=bins,align='left',histtype='step' )
plt.vlines(x=avrg[qb],ymin=0,ymax=max(count))
# Min,Max=val[0],val[-1]
# ticks=np.arange(Min,Max,step=.01)
# ll=['%.3f' %a for a in val]
plt.xticks(np.arange(start, stop+step, 2*step))
# plt.xticks(ticks=val[::18],labels=ll[::18])
plt.xlabel('Probability of discovering Eve when using %i check-bits' %N[qb],fontsize=12)
plt.ylabel('Frequency',fontsize=12)
plt.xlim(0.01,1.0)
plt.title('BB84 protocol using %i qubits'%n)

plt.savefig(outpath+'/'+'BB84-dist with %i check-bits.png'%N[qb],dpi=200)
plt.show()
plt.close()

## For all plots individually

In [0]:
for qb in range(len(N)):
    count,val=[],[]
#     plt.figure(num=qb,figsize=(16,9))
    plt.figure(num=qb,dpi=300)
    count,val,_=plt.hist(dist[:,qb],bins=bins,align='left',histtype='step' )
    plt.xticks(np.arange(start, stop+step, 2*step))
    plt.xlabel('Probability of discovering Eve when using %i check-bits' %N[qb])
    plt.ylabel('Frequency')
    plt.xlim(0.01,1.0)
    plt.title('BB84 protocol using %i qubits'%n)
    plt.savefig(outpath+'/'+'BB84-dist with %i check-bits'%N[qb],dpi=200)
    plt.show()
    plt.close()

## A Closer look

In [0]:
for qb in range(len(N)):
    count,val=[],[]
    plt.figure(num=qb,figsize=(16,9))
    count,val,_=plt.hist(dist[:,qb],bins=50,align='left',histtype='step' )
    ll=['%.3f' %a for a in val]
    plt.xticks(ticks=val[::3],labels=ll[::3])
    plt.xlabel('Probability of discovering Eve when using %i check-bits' %N[qb])
    plt.ylabel('Frequency')
#     plt.xlim(0.1,1)
    plt.title('BB84 protocol using %i qubits'%n)
    plt.savefig(outpath+'/'+'CloserLook_BB84-dist with %i check-bits'%N[qb],dpi=200)
    plt.show()
    plt.close()

## A few of them together

In [0]:
plt.figure()
start = 0
stop  = 1
step  = .05
bins=np.linspace(start, stop, num=250)

for qb in range(len(N)):
    count,val,_=plt.hist(dist[:,qb],align='left',histtype='step',label='Probability using %i check-bits' %N[qb],bins=bins )
plt.vlines(x=avrg,ymin=0,ymax=max(count), colors='k', linestyles='dashed',  label='Average Values',alpha=.63)
plt.xticks(np.arange(start, stop+step, 2*step),fontsize=12)
plt.yticks(fontsize=15)
plt.xlabel('Probability of discovering Eve',fontsize=15)
plt.ylabel('Frequency',fontsize=15)
plt.xlim(0.05,1.0)
# plt.grid(axis='x')
plt.legend(shadow=True,fontsize=7,bbox_to_anchor=(1.45,1), loc="upper right",mode='expand')
plt.title('BB84 protocol using %i qubits'%n , fontsize=20)
plt.savefig(outpath+'/'+'BB84-dist-superimposed',dpi=200,bbox_inches="tight")
plt.show()
plt.close()

In [0]:
# print(p)
# N1=np.arange(17)
P=1-(.75)**N
print(P)
# print(np.e)

## Errors between our theoretical values and our simulation values for the probabilities of detecting Eve

In [0]:
# Root Squared error
rse=np.sqrt((avrg-P)**2)
# print('root squared error: \n',rse)

# Difference 
Error=avrg-P
# print("Error: \n",Error)

# Absolute Error
abserr=np.abs(avrg-P)
print('absolute error: \n',abserr)
# print('\nIs root squared error same as absolute error?:\n', rse==abserr)


# Percentage Error
percenterr=(abserr/P)
print('\nPercentage of Error')
for i in percenterr:
    print('%.4f' %i+' %')
    
# print(percenterr)

In [0]:
# for i in range(len(N)):
# count,val=[],[]
plt.figure(figsize=(16,9))
plt.bar(N,avrg,alpha=.5,align='edge')
# plt.plot(N,avrg,'ok',label='Simulation')
plt.plot(N,P,'--g',label='$P=1-(3/4)^N$')
plt.xticks(ticks=N,fontsize=14)
plt.yticks(ticks=np.arange(start,stop+step,2*step),fontsize=14)
plt.xlabel('Number of Check-bits',fontsize=14)
plt.ylabel('Average Prob of Discovering Eve',fontsize=14)
plt.title('BB84 \n Probability of Discovering Eve when varying amount of check-bits', fontsize=20)
plt.legend(fontsize=14,loc='upper center',shadow=True)
plt.grid(axis='y',color='k',linestyle='--',alpha=.7)
plt.savefig(outpath+'/'+'BB84-prob-per-Check-bits.png',dpi=200,format='png')
plt.show()
plt.close()

# We also have a CSV file with data from a more precise simulation

We won't need to run a sim each time you need to look a the plots or generate other plots

We can use pandas to read the CSV file provided in the Github Repo

In [0]:
import pandas as pd

## Configuration of the plots

In [0]:
df=pd.read_csv('Distribution-Data-for-BB84.csv')
df

In [0]:
print(df.info())
df.describe()

In [0]:
#Where to store the plots
outpath='plots_BB84'

#Check if folder exists
if outpath not in os.listdir():
    os.mkdir(outpath)
else: 
    print(outpath,'already exists!')


#Configuration for the plots
start = 0
stop  = 1
step  = .05
bins=np.linspace(start, stop, num=250)
    

## All Checkbit distributions

In [0]:
df.plot(figsize=(16,8),kind='hist',fontsize=14, align='left',histtype='stepfilled' ,bins=bins)
plt.legend(loc='upper center',ncol=5,fontsize=13)
plt.ylabel('Frequency',fontsize=15)
plt.xlabel('Probability of Detecting Eve', fontsize=15)
plt.xticks(np.arange(start, stop+step, step))
plt.xlim(0.05,1.0)
plt.title('BB84 \n Probability of Detecting Eve varying the amount of checkbits',fontsize=20)
plt.savefig(outpath+'/'+'Pandas-Dist-Supermposed-All.png',dpi=200,format='png')
plt.show()

## Testing


In [0]:
avrg=df.mean()
avrg

In [0]:
df[df.columns[qb]]

In [0]:


plt.figure(figsize=(16,8))
for qb in range(5):
#     if qb==2**a[-1]:
#         count,val,_=plt.hist(df["15"],align='left',histtype='bar',label='Probability using 15 check-bits' ,bins=bins )
#     else:
    count,val,_=plt.hist(df[df.columns[qb]],align='left',histtype='bar',label='Probability using %s check-bits' %df.columns[qb],bins=bins )

plt.vlines(x=avrg,ymin=0,ymax=max(count), colors='k', linestyles='dashed',  label='Average Values',alpha=.63)
# plt.vlines(x=avrg,ymin=0,ymax=max(count), colors='k', linestyles='dashed',alpha=.63)
plt.xticks(np.arange(start, stop+step, step),fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Probability of Detecting Eve',fontsize=15)
plt.ylabel('Frequency',fontsize=15)
plt.xlim(0.05,1.0)
# plt.grid(axis='x')
plt.legend(loc='upper center',ncol=2,shadow=True,fontsize=12)
plt.title('BB84\n Probability of Detecting Eve' , fontsize=20)
plt.savefig(outpath+'/'+'Pandas-BB84-dist-superimposed.png',dpi=200,format='png')
plt.show()
plt.close()

## End testing

## Some of the Distributions and their average values

In [0]:
plt.figure(figsize=(16,8))
for qb in 2**a:
    if qb==2**a[-1]:
        count,val,_=plt.hist(df["15"],align='left',histtype='bar',label='Probability using 15 check-bits' ,bins=bins )
    else:
        plt.hist(df['%s'%qb],align='left',histtype='bar',label='Probability using %i check-bits' %qb,bins=bins )

plt.vlines(x=avrg[2**a[:-1]-1],ymin=0,ymax=max(count), colors='k', linestyles='dashed',  label='Average Values',alpha=.63)
plt.vlines(x=avrg[-1],ymin=0,ymax=max(count), colors='k', linestyles='dashed',alpha=.63)
plt.xticks(np.arange(start, stop+step, step),fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Probability of Detecting Eve',fontsize=15)
plt.ylabel('Frequency',fontsize=15)
plt.xlim(0.05,1.0)
# plt.grid(axis='x')
plt.legend(loc='upper center',ncol=2,shadow=True,fontsize=12)
plt.title('BB84\n Probability of Detecting Eve' , fontsize=20)
plt.savefig(outpath+'/'+'Pandas-BB84-dist-superimposed.png',dpi=200,format='png')
plt.show()
plt.close()